# Lab 9: SLM-Powered Interpretation of Apriori-Based Association Rules 

```bash
# install required packages
!pip install mlxtend transformers
```

## Step 1: Import and Prepare Transaction Data 

In [2]:
# import libraries
import pandas as pd 
from mlxtend.preprocessing import TransactionEncoder 

In [3]:
dataset = [ 
['Milk', 'Bread', 'Butter'], 
['Milk', 'Bread'], 
['Milk', 'Coke'], 
['Bread', 'Coke'], 
['Bread', 'Butter'] 
] 

te = TransactionEncoder() 
te_data = te.fit(dataset).transform(dataset) 
df = pd.DataFrame(te_data, columns=te.columns_) 

## Step 2: Apply Apriori Algorithm with Lower Thresholds

In [4]:
from mlxtend.frequent_patterns import apriori, association_rules 

frequent_itemsets = apriori(df, min_support=0.2, use_colnames=True) 
rules = association_rules(frequent_itemsets, metric="confidence", 
min_threshold=0.3) 

print(" Frequent Itemsets:\n", frequent_itemsets) 
print("\n Association Rules:\n", rules)

 Frequent Itemsets:
    support               itemsets
0      0.8                (Bread)
1      0.4               (Butter)
2      0.4                 (Coke)
3      0.6                 (Milk)
4      0.4        (Bread, Butter)
5      0.2          (Coke, Bread)
6      0.4          (Bread, Milk)
7      0.2         (Milk, Butter)
8      0.2           (Coke, Milk)
9      0.2  (Milk, Bread, Butter)

 Association Rules:
         antecedents      consequents  antecedent support  consequent support  \
0           (Bread)         (Butter)                 0.8                 0.4   
1          (Butter)          (Bread)                 0.4                 0.8   
2            (Coke)          (Bread)                 0.4                 0.8   
3           (Bread)           (Milk)                 0.8                 0.6   
4            (Milk)          (Bread)                 0.6                 0.8   
5            (Milk)         (Butter)                 0.6                 0.4   
6          (Butter)    

## Step 3: Use SLM (Small Language Model) to Explain Rules 

Using Gemma-3 by Google

In [5]:
# configure api
from dotenv import load_dotenv
import os

load_dotenv()
gemini_api_key = os.getenv("GEMINI_API_KEY")

In [6]:
from google import genai

client = genai.Client(api_key=gemini_api_key)

model = "gemma-3-27b-it"

## Step 4: Generate Explanations

In [ ]:
if not rules.empty: 
    for index, row in rules.iterrows(): 
        antecedent = ', '.join(list(row['antecedents'])) 
        consequent = ', '.join(list(row['consequents'])) 
        rule = f"If someone buys {antecedent}, they are likely to buy {consequent}." 
 
        print("\n Rule:", rule) 
        explanation = client.models.generate_content(model = model, contents = f"Explain this rule: {rule}") 
        print("SLM Explanation:", explanation.text) 
else: 
        print("\n No association rules found. Try further lowering support/confidence thresholds or check your data.")


 Rule: If someone buys Bread, they are likely to buy Butter.


TypeError: 'GenerateContentResponse' object is not subscriptable